# Mixed poisson

Solving $-\Delta p = f$, with $\nabla p = u$.

In [1]:
import dolfinx
import dolfinx.fem
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import ufl

In [2]:
n = 20
m = 25
domain = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, m)

In [3]:
tdim = domain.topology.dim
fdim = tdim - 1

num_cells = domain.topology.index_map(tdim).size_local
num_cells

1000

In [4]:
boundary_entities = dolfinx.mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full((x.shape[1], ), True))
boundary_entities

array([   0,    2,    5,    7,   14,   16,   26,   28,   41,   43,   59,
         61,   80,   82,  104,  106,  131,  133,  161,  163,  194,  196,
        230,  232,  269,  271,  311,  313,  356,  358,  404,  406,  455,
        457,  509,  511,  566,  568,  626,  628,  631,  689,  691,  750,
        752,  811,  813,  872,  874,  933,  935,  992,  994, 1048, 1050,
       1101, 1103, 1151, 1153, 1198, 1200, 1242, 1244, 1283, 1285, 1321,
       1323, 1356, 1358, 1388, 1390, 1417, 1419, 1443, 1445, 1466, 1468,
       1486, 1488, 1503, 1505, 1517, 1519, 1528, 1530, 1536, 1538, 1541,
       1543, 1544], dtype=int32)

In [5]:
from ufl import (FiniteElement, Measure, VectorElement, MixedElement, SpatialCoordinate,
                 TestFunctions, TrialFunctions, div, exp, inner, FacetNormal)
from dolfinx import fem, io, mesh

In [6]:
k = 1
# V_el = FiniteElement("RT", domain.ufl_cell(), k)
# V_el = VectorElement("CG", domain.ufl_cell(), k)
V_el = VectorElement("CR", domain.ufl_cell(), k)
Q_el = FiniteElement("DG", domain.ufl_cell(), k - 1)
W_el = MixedElement([V_el, Q_el])
W = fem.FunctionSpace(domain, W_el)
V, _ = W.sub(0).collapse()
Q, _ = W.sub(1).collapse()

In [7]:
uD = fem.Function(V)
pD = fem.Function(Q)

x = SpatialCoordinate(domain)

p_ex = ufl.sin(ufl.pi*2*x[0])*ufl.sin(ufl.pi*2*x[1])
u_ex = ufl.grad(p_ex)

uD.interpolate(fem.Expression(u_ex, V.element.interpolation_points()))
pD.interpolate(fem.Expression(p_ex, Q.element.interpolation_points()))

g = -ufl.div(ufl.grad(p_ex))

In [8]:
(u, p) = TrialFunctions(W)
(v, q) = TestFunctions(W)

dx = Measure("dx", domain)
ds = Measure("ds", domain)
n = ufl.FacetNormal(domain)
# dx = ufl.dx
# ds = ufl.ds
a  = inner(u, v) * dx + inner(p, div(v)) * dx 
a += inner(div(u), q) * dx
L = -inner(g, q) * dx + ufl.inner(pD*v,n) * ds 

In [9]:
# # Now setup boundary conditions
# def bc_function(x):
#     values = np.zeros((2, x.shape[1]))
#     values[1, :] = np.sin(5 * x[0])
#     values[0, :] = np.cos(5 * x[0])
#     return values

# gh = fem.Function(Q)
# gh.interpolate(bc_function)

# boundary_dofs = dolfinx.fem.locate_dofs_topological((V.sub(0), Q), fdim, boundary_entities)
# bc = dolfinx.fem.dirichletbc(gh, boundary_dofs)

In [10]:
# problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={
#                                   "ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})

In [11]:
problem = fem.petsc.LinearProblem(a, L, bcs=[], petsc_options={
                                   "ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})

In [12]:
try:
    w_h = problem.solve()
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

u_h, p_h = w_h.split()

In [13]:
u_h.name = 'u'
p_h.name = 'p'

In [14]:
from dolfinx import io
# with io.VTXWriter(domain.comm, "output.bp", [w_h]) as vtx:
#     vtx.write(0.0)
with io.XDMFFile(domain.comm, "poisson.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(u_h)
    xdmf.write_function(p_h)

In [15]:
import numpy as np

error_form_u = fem.form(inner(u_h-uD, u_h-uD) * dx)
error_local_u = fem.assemble_scalar(error_form_u)
errorL2_u = np.sqrt(domain.comm.allreduce(error_local_u, op=mpi4py.MPI.SUM))

error_form_p = fem.form(inner(p_h-pD, p_h-pD) * dx)
error_local_p = fem.assemble_scalar(error_form_p)
errorL2_p = np.sqrt(domain.comm.allreduce(error_local_p, op=mpi4py.MPI.SUM))

if domain.comm.rank == 0:
    print(fr"$L^2$-error u: {errorL2_u:.2e}")
    print(fr"$L^2$-error p: {errorL2_p:.2e}")

$L^2$-error u: 3.25e+00
$L^2$-error p: 2.48e-01
